Before you turn this problem in, make sure everything runs as expected. **Restart the kernel and run all cells** (in the menubar, select Kernel$\rightarrow$Restart & Run All).

Make sure you fill in any place that says `YOUR CODE HERE ???` or "YOUR ANSWER HERE". Code cells are autograded, and if the test cells don't run all the way to the line that  prints "Tests Passed (# points)", you will not recieve any point for that test.

# CSCI 3155: Assignment 9

Topics: 
- Type Inference
- Basics of Objects
- Variance Annotations.

Readings: Notes posted on moodle.  Chapters from Odersky

- Chapter 4 (revise)
- Chapter 10
- Chapter 11 (take note of Any, AnyVal, Null, Nothing)
- Chapter 12 
- Chapter 19

__Name__: Derek Thomas

In [1]:
// TEST HELPER
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"\n*** Tests Passed (1 point) ***\n")
    else print(s"\n*** Tests Passed ($points points) ***\n")
}

defined function passed

## Problem 1 (20 points): Objects in Scala

### A (12 points)
Consider the class below for an employee record in an organization that has three fields `name`, `eid` (employee id) and `salary`.

1) As given, scala will complain that class employee cannot extend the abstract class People. Add the missing methods to enable the class Employee to properly extend People

2) Write a factory pattern that takes in a formatted string of the form "John Doe,29585,33110" consisting of a string and two numbers, to extract the name and salaries for each employee. You may want to use `scala.util.matching.Regex` [https://www.scala-lang.org/api/2.9.2/scala/util/matching/Regex.html]
If the input fails to match the pattern, an `IllegalArgumentException` must be thrown

Companion objects are covered in scala book Chapter 4.3. Also lookup here: https://alvinalexander.com/scala/factory-pattern-in-scala-design-patterns

In [2]:
import scala.util.matching.Regex

abstract class People {
    /* I need the ability to convert every person to a string */
    def toString: String
    /* I need the ability to check if two people are the same */
    def equals (p: People): Boolean
}

/* 
  Note that for an employee to be equal to an object of type People, that object
  must be an employee with matching eid number 
*/

class Employee( val name: String, val eid: Int, var salary: Int) extends People {
    // YOUR CODE HERE
    override def toString = name
    
    def equals(p: People) = {
        if (p.isInstanceOf[Employee]) {
            p.asInstanceOf[Employee].eid == eid
        }
        else {
            false
        }
    }
}

object Employee {
    def apply(formattedInput: String): Employee = {
        /* 
        Input format must be 
        (full name with possible spaces)[optional whitespaces],[optional whitespaces](employee ID number)[optional whitespaces],[optional whitespaces](employee salary)
        */
        // YOUR CODE HERE
        val employee = """([a-zA-Z\s]+?)\s*,\s*(\d+)\s*,\s*(\d+)""".r
        formattedInput match {
            case employee(name, eid, salary) => new Employee(name, eid.toInt, salary.toInt)
            case _ => throw new IllegalArgumentException
        }
    }
}

import scala.util.matching.Regex


defined class People
defined class Employee
defined object Employee

In [3]:
//BEGIN TEST
val e1 = Employee("Jonathan Doe, 12893, 39813")
println(e1.toString)
assert(e1.name == "Jonathan Doe", "Failed")
assert(e1.eid == 12893, "Failed")
assert(e1.salary == 39813, "Failed")
assert(e1.equals(new Employee("whoknows", 12893, 39813)))
passed(4)
//END TEST

Jonathan Doe

*** Tests Passed (4 points) ***


e1: Employee = Jonathan Doe

In [4]:
//BEGIN TEST
try {
    val e2 = Employee("Badly Formatted String, 29812, $3102")
    assert(false, "Failed, the string is badly formatted but your code is OK with it.")
} catch {
    case e => println(s"Expected behavior seen")
}
passed(4)
//END TEST

Expected behavior seen

*** Tests Passed (4 points) ***


In [5]:
//BEGIN TEST
val e1 = Employee("Janet Maria Catherine Doe    , 12894   , 121331")
println(e1.toString)
assert(e1.name == "Janet Maria Catherine Doe", "Failed")
assert(e1.eid == 12894, "Failed")
assert(e1.salary == 121331, "Failed")
assert(e1.equals(new Employee("random", 12894, 121332)))
passed(4)
//END TEST

Janet Maria Catherine Doe

*** Tests Passed (4 points) ***


e1: Employee = Janet Maria Catherine Doe

### B (8 points)
We wish to add a new class of people to our system. These are "Contractor" with the information associated should involve their name (string), contract ID (Integer) and contract supervisor (Employee).

Write a new class named `Contractor` whose constructor should take in the name of the person (String), their ID number (Integer) and Supervisor (Employee) and extend from People. 

A contractor object is equal to an object p of type People, only if p is actually an instance of contractor, and if their names are the same and they have the same contract ID number.

In [6]:
// YOUR CODE HERE
class Contractor( val name: String, val id: Int, var supervisor: Employee) extends People {
    override def toString = name
    def equals(p: People) = {
        if (p.isInstanceOf[Contractor]) {
            p.asInstanceOf[Contractor].name == name && p.asInstanceOf[Contractor].id == id
        }
        else {
            false
        }
    }
}

defined class Contractor

In [7]:
//BEGIN TEST
val e1 = Employee("Jonathan Doe, 12893, 39813")
val c1 = new Contractor("John Contract Worker", 12093, e1)
assert( !c1.equals(e1), "Failed")
assert( c1.equals(c1), "Failed")
passed(4)
//END TEST


*** Tests Passed (4 points) ***


e1: Employee = Jonathan Doe
c1: Contractor = John Contract Worker

In [8]:
//BEGIN TEST
val e1 = Employee("Jonathan Doe, 12893, 39813")
val e2 = Employee("Jane Doe, 12894, 43814")
val c1 = new Contractor("John Contract Worker", 12093, e2)
val c2 = new Contractor("John Contract Worker", 12093, e1)
assert( c1.equals(c2), "Failed")
passed(4)
//END TEST


*** Tests Passed (4 points) ***


e1: Employee = Jonathan Doe
e2: Employee = Jane Doe
c1: Contractor = John Contract Worker
c2: Contractor = John Contract Worker

## Problem 2 (40 points): Traits in Scala

For this problem, we have defined two traits: `NumberOfLegs` that helps us define how many legs a given animal has and `WarmBlooded` that applies to warm blooded animals.

In [2]:
abstract class Animal 

trait NumberOfLegs {
    val nLegs: Int
    def getNumberOfLegs: Int = nLegs
}

trait WarmBlooded extends Animal {
    val bodyTempMaintained: Double
    def getBodyTemp: Double = bodyTempMaintained 
}

defined class Animal
defined trait NumberOfLegs
defined trait WarmBlooded

### A (5 points)

Define a class `Human` that inherits from `Animal` and mixes in the traits `NumberOfLegs` with `nLegs = 2` and `WarmBlooded` with `bodyTempMaintained = 98`. The class `Human` must take in a parameter called `name` of type `String` and implement a `getName` method without any parameters.

In [10]:
// YOUR CODE HERE
class Human(val name: String, val bodyTempMaintained: Double = 98, val nLegs: Int = 2) extends WarmBlooded with NumberOfLegs {
    def getName = name
}

defined class Human

In [11]:
//BEGIN TEST
val t1 = new Human("Jane Smith")
assert(t1.getNumberOfLegs == 2, "Your human does not have two legs")
assert(t1.bodyTempMaintained == 98.0, "Your human does not maintain a body temp of 98")
assert(t1.getBodyTemp == 98.0, "Your human's getBodyTemp Function is not working")
assert(t1.getName == "Jane Smith", "Your human's name is not setting correctly")
passed(5)
//END TEST


*** Tests Passed (5 points) ***


t1: Human = ammonite.$sess.cmd9$Helper$Human@6add6aab

### B (5 points)
Define a class named `Table` that mixes in the trait `NumberOfLegs` with `nLegs = 4`. 

_Note:_ `Table` cannot mixin the trait `WarmBlooded` (make sure you understand why that is as a quick check of concepts).

In [12]:
// YOUR CODE HERE
class Table(val nLegs: Int = 4) extends NumberOfLegs

defined class Table

In [13]:
//BEGIN TEST
val tbl = new Table()
assert(tbl.getNumberOfLegs == 4, "A Table must have four legs")
passed(5)
//END TEST


*** Tests Passed (5 points) ***


tbl: Table = ammonite.$sess.cmd11$Helper$Table@6e3ea62a

### C (15 points)
Define a class called `Menagerie` with type parameter `T` such that 

- T must always be a derived type of Animal with the trait NumberOfLegs (In scala you can say `T <: A with B` to indicate that a type parameter T must derive from class A with trait B applied).

- `Menagerie` has a list of objects of type `T`.

- The user must be able to instantiate an object of type Menagerie without providing any class parameters: `new Menagerie[T]`

- The class Menagerie has the method `addAnimalToMenagerie` that adds an object of type `T` to menagerie and returns a new Menagerie.

- Implement a method `addMultipleAnimalsToMenagerie` that adds more than one animal. Lookup syntax below.
 
 
Lookup the syntax for varargs or repeated parameters (T*) here: https://alvinalexander.com/scala/how-to-define-methods-variable-arguments-varargs-fields. Also in Chapter 8.8 of Odersky book.


- Add a method `totalLegs: Int` that sums up how many legs the animals in the menagerie have in total.

~~~
class Menagerie[T...] () {
   // ... Extra Stuff that you may want to add ...
   def addAnimalToMenagerie (a: T) : Menagerie[T]
   def addMultipleAnimalsToMenagerie(animals: T*): Menagerie[T]
   def totalLegs:Int
}
~~~


In [10]:
// YOUR CODE HERE
class Menagerie[T <: Animal with NumberOfLegs] (val occupents: List[T]) {
    def this() {
        this(Nil)
    }
    def addAnimalToMenagerie(animal: T) = {
        animal :: occupents
        println(occupents)
        new Menagerie(occupents)
    }
    def addMultipleAnimalsToMenagerie(animals: T*) = new Menagerie(animals.toList ++ occupents)
    def totalLegs: Int = occupents.map(_.getNumberOfLegs).sum
}

defined class Menagerie

In [11]:
//BEGIN TEST SETUP
class Donkey extends Animal with NumberOfLegs with WarmBlooded {
    val nLegs:Int = 4
    val bodyTempMaintained = 96.0
}

class Lion extends Animal with NumberOfLegs {
    val nLegs:Int = 4
}

class Mule extends Donkey

val d1 = new Donkey()
val d2 = new Donkey()
val d3 = new Donkey()
val d4 = new Mule()
//END TEST SETUP

defined class Donkey
defined class Lion
defined class Mule
d1: Donkey = ammonite.$sess.cmd10$Helper$Donkey@53462ce9
d2: Donkey = ammonite.$sess.cmd10$Helper$Donkey@7d404bc6
d3: Donkey = ammonite.$sess.cmd10$Helper$Donkey@28eac1b4
d4: Mule = ammonite.$sess.cmd10$Helper$Mule@4f4a9e9e

In [12]:
//BEGIN TEST
val m1 = new Menagerie[Donkey]
passed(3)
//END TEST


*** Tests Passed (3 points) ***


m1: Menagerie[Donkey] = ammonite.$sess.cmd9$Helper$Menagerie@18ef02bf

In [13]:
//BEGIN TEST
val m1 = new Menagerie[Donkey]
val m2 = m1.addAnimalToMenagerie(d1)
val n = m2.totalLegs
assert(n == 4, s"1 quadraped = 4 legs. You have $n legs : are you missing legs? Too many legs?")
passed(6)
//END TEST

List()


: 

In [14]:
//BEGIN TEST
val m1 = new Menagerie[Donkey]
val m2 = m1.addMultipleAnimalsToMenagerie(d1, d2, d3, d4)
val n = m2.totalLegs
assert(n == 16, s"4 quadrapeds = 16 legs. You have $n legs : are you missing legs? Too many legs?")
passed(6)
//END TEST


*** Tests Passed (6 points) ***


m1: Menagerie[Donkey] = ammonite.$sess.cmd9$Helper$Menagerie@d7fcb53
m2: Menagerie[Donkey] = ammonite.$sess.cmd9$Helper$Menagerie@6a09212b
n: Int = 16

### D (15 points)
Currently a `Menagerie` as defined above is Invariant. Make it covariant so that  subtyping relations such as `Menagerie[Mule] <: Menagerie[Donkey] <: Menagerie[Animal]`.

In [19]:
// YOUR CODE HERE
class Menagerie[+T <: Animal with NumberOfLegs] (val occupents: List[T]) {
    def this() {
        this(Nil)
    }
    def addAnimalToMenagerie[U >: T <: Animal with NumberOfLegs](a: U) = new Menagerie[U](a :: occupents)
    def addMultipleAnimalsToMenagerie[U >: T <: Animal with NumberOfLegs](animals: U*) = new Menagerie[U](animals.toList ++ occupents)
    def totalLegs: Int = occupents.map(_.getNumberOfLegs).sum
}

defined class Menagerie

In [20]:
//BEGIN TEST 
class Donkey extends Animal with NumberOfLegs with WarmBlooded {
    val nLegs:Int = 4
    val bodyTempMaintained = 96.0
}

class Mule extends Donkey

val d1 = new Donkey()
val d2 = new Donkey()
val d3 = new Donkey()
val mu1 = new Mule()
val m1 = new Menagerie[Donkey]
// Inheritance among Menageries:
val m2: Menagerie[Donkey] = new Menagerie[Mule]
// Strongest type selection:
val m3: Menagerie[Donkey] = m1.addMultipleAnimalsToMenagerie(d1, d2, d3, mu1)

print("Successfully added Lions and Donkeys to the same menagerie!")

passed(15)
//END TEST

Successfully added Lions and Donkeys to the same menagerie!
*** Tests Passed (15 points) ***


defined class Donkey
defined class Mule
d1: Donkey = ammonite.$sess.cmd19$Helper$Donkey@4fe1775
d2: Donkey = ammonite.$sess.cmd19$Helper$Donkey@394076cd
d3: Donkey = ammonite.$sess.cmd19$Helper$Donkey@7be613d7
mu1: Mule = ammonite.$sess.cmd19$Helper$Mule@10722771
m1: Menagerie[Donkey] = ammonite.$sess.cmd18$Helper$Menagerie@5174f56b
m2: Menagerie[Donkey] = ammonite.$sess.cmd18$Helper$Menagerie@7cd95bb8
m3: Menagerie[Donkey] = ammonite.$sess.cmd18$Helper$Menagerie@7b340dd5